# **Import Library**
Bagian `import re, numpy as np, pandas as pd` dan import lain memanggil pustaka yang dipakai sepanjang proses preprocessing. Pandas digunakan untuk mengelola dataset, NumPy membantu operasi numerik, sedangkan NLTK dan Sastrawi menangani fungsi khusus pemrosesan teks.

In [1]:
!pip install -q nltk Sastrawi
import re, numpy as np, pandas as pd
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


# **Inisialisasi Stopwords dan Stemmer**
Kode `nltk.download('stopwords')` dan blok try–except membuat daftar STOPWORDS bahasa Indonesia, dengan cadangan bahasa Inggris jika korpus Indonesia tidak tersedia. Daftar ini kemudian ditambah kata-kata gaul umum, dan objek STEMMER dibuat dari Sastrawi untuk keperluan stemming kata jika dibutuhkan pada tahap berikutnya.





In [2]:
nltk.download('stopwords')
try:
    STOPWORDS = set(stopwords.words('indonesian'))
except OSError:
    STOPWORDS = set(stopwords.words('english'))
    STOPWORDS.update([
        "tidak","ya","enggak","nggak","gak","kagak","kok","sih","banget","aja","lah","dong",
        "nih","deh","loh","pun","kan","kayak","kayaknya","jadi","masih","udah","udahh",
        "bener","sekali","bangett"
    ])

STEMMER = StemmerFactory().create_stemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Pemanggilan Dataset**
Bagian `CSV_PATH` mendefinisikan lokasi file dan memuat data ulasan Traveloka ke dalam sebuah dataframe. Langkah ini menjadi pintu masuk utama supaya seluruh proses pembersihan teks bekerja pada data yang sama.

In [6]:
CSV_PATH = "/content/data_traveloka_gabungan.csv"
df = pd.read_csv(
    CSV_PATH,
    engine="python",      # pakai python engine
    on_bad_lines="skip"   # lewati baris yang rusak
)

# **Validasi Kolom Teks**
Potongan `TEXT_COL = "content"` diikuti pengecekan `if TEXT_COL not in `df.columns memastikan bahwa kolom yang akan diproses memang bernama content dan tersedia di dataset. Jika kolom tidak ditemukan, kode menghentikan proses dengan pesan kesalahan yang jelas agar pengguna segera memperbaiki nama kolom.

In [7]:
TEXT_COL = "content"
if TEXT_COL not in df.columns:
    raise ValueError("Kolom 'content' tidak ada di dataframe")

# **Kamus Slang**

In [8]:
SLANG = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "hubla": "perhubungan laut", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}


# **Definisi Pola Regex**
Bagian kode yang berisi `URL_RE, MENTION_RE, HASHTAG_RE, HTML_RE, EMOJI_RE, REPEAT_RE, NON_ALPHA_RE, dan MULTISPACE_RE` digunakan untuk mendefinisikan pola pembersihan teks. Masing-masing pola bertugas menghapus elemen yang tidak relevan, seperti tautan, mention, hashtag, tag HTML, emoji, karakter non huruf, pengulangan huruf berlebih, dan spasi ganda. Dengan cara ini, teks ulasan yang masuk ke proses berikutnya sudah jauh lebih bersih dan seragam.

In [9]:
URL_RE        = re.compile(r"https?://\S+|www\.\S+", re.I)
MENTION_RE    = re.compile(r"@\w+")
HASHTAG_RE    = re.compile(r"#\w+")
HTML_RE       = re.compile(r"<[^>]+>")
EMOJI_RE      = re.compile("[" "\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF"
                           "\U0001F680-\U0001F6FF" "\U0001F1E0-\U0001F1FF"
                           "\U00002700-\U000027BF" "\U000024C2-\U0001F251" "]+",
                           flags=re.UNICODE)
REPEAT_RE     = re.compile(r"(.)\1{2,}", re.DOTALL)
NON_ALPHA_RE  = re.compile(r"[^a-zA-Z\u00C0-\u02AF\u1E00-\u1EFF\s]")
MULTISPACE_RE = re.compile(r"\s+")

# **Fungsi Pembersihan Teks**
Fungsi `normalize_slang, remove_stopwords, dan stem_id` disiapkan sebagai fungsi bantu untuk menormalkan kata slang, membuang stopwords, dan melakukan stemming jika diperlukan. Seluruh fungsi bantu tersebut kemudian dipanggil di dalam `clean_text`, yang menjalankan proses pembersihan secara berurutan mulai dari mengubah huruf menjadi lowercase, menghapus pola regex, menormalkan slang, hingga merapikan spasi. Hasil akhirnya adalah teks ulasan yang sudah rapi dan terstandardisasi sehingga siap digunakan pada tahap preprocessing selanjutnya.

In [10]:
def normalize_slang(t, m):    return " ".join(m.get(w, w) for w in t.split())
def remove_stopwords(t):      return " ".join([w for w in t.split() if w not in STOPWORDS and len(w) > 1])
def stem_id(t):               return STEMMER.stem(t)

def clean_text(t):
    if not isinstance(t, str):
        t = str(t)
    t = t.lower()
    t = URL_RE.sub(" ", t)
    t = MENTION_RE.sub(" ", t)
    t = HASHTAG_RE.sub(" ", t)
    t = HTML_RE.sub(" ", t)
    t = EMOJI_RE.sub(" ", t)
    t = normalize_slang(t, SLANG)
    t = REPEAT_RE.sub(r"\1\1", t)
    t = NON_ALPHA_RE.sub(" ", t)
    t = MULTISPACE_RE.sub(" ", t).strip()
    return t

# **Preprocessing**
Tahap ini mendefinisikan fungsi preprocess sebagai pipeline utama pembersihan teks dan langsung menerapkannya pada kolom content. Di dalam fungsi, data terlebih dahulu diubah ke tipe string, nilai kosong seperti “nan” dan “None” dihapus, lalu setiap baris teks dibersihkan dengan clean_text, kemudian (sesuai parameter) dapat dibuang stopwords dan bisa juga dilakukan stemming. Setelah itu, teks yang terlalu pendek disaring, hasilnya disusun menjadi DataFrame yang berisi row_id, original_text, dan clean_text, serta ditambah fitur tokens, n_tokens (jumlah kata), dan n_chars (jumlah karakter). Di bagian akhir, fungsi preprocess dipanggil dengan do_stopwords=True dan do_stem=False sehingga teks sudah bersih dan bebas stopwords, tetapi bentuk katanya tetap utuh, lalu disimpan dalam variabel pre_df sebagai data siap pakai untuk analisis berikutnya.

In [11]:
def preprocess(series,
               dropna=True,
               drop_duplicates=True,
               min_len=3,
               do_stopwords=True,
               do_stem=False):
    s = series.astype(str).copy()
    if dropna:
        s = s.replace(["nan","None"], np.nan).dropna()
    cleaned = s.apply(clean_text)
    if do_stopwords:
        cleaned = cleaned.apply(remove_stopwords)
    if do_stem:
        cleaned = cleaned.apply(stem_id)
    cleaned = cleaned[cleaned.str.len() >= min_len]

    out = pd.DataFrame({
        "row_id": cleaned.index,
        "original_text": s.loc[cleaned.index],
        "clean_text": cleaned
    })
    if drop_duplicates:
        out = out.drop_duplicates(subset=["clean_text"])

    out["tokens"]   = out["clean_text"].str.split()
    out["n_tokens"] = out["tokens"].apply(len)
    out["n_chars"]  = out["clean_text"].str.len()
    return out.reset_index(drop=True)

pre_df = preprocess(df[TEXT_COL], do_stopwords=True, do_stem=False)

# **Penggabungan Metadata dan Label**
Daftar `want` berisi nama-nama kolom penting dari dataset asli, seperti reviewId, userName, content, score, dan appVersion. Daftar ini difilter menjadi `base_cols` agar hanya kolom yang memang tersedia di dataframe yang diambil. Data hasil preprocessing (`pre_df`) kemudian digabung dengan kolom-kolom tersebut berdasarkan indeks baris (`row_id`) sehingga teks yang sudah bersih tetap memiliki informasi metadata lengkap. Setelah itu, kode mengecek apakah ada kolom label sentimen (misalnya “label”, “sentiment”, atau “polarity”); jika ada, kolom tersebut ikut ditambahkan ke `pre_df`.

In [12]:
want = [
    "Unnamed: 0","reviewId","userName","userImage","content","score",
    "thumbsUpCount","reviewCreatedVersion","at","replyContent","repliedAt","appVersion"
]
base_cols = [c for c in want if c in df.columns]

pre_df = pre_df.merge(df[base_cols], left_on="row_id", right_index=True, how="left")

# Label jika ada
label_col = next((c for c in ["label","sentiment","polarity","target","y"] if c in df.columns), None)
if label_col:
    pre_df["label"] = df.loc[pre_df["row_id"], label_col].values


# **Pengaturan Urutan Kolom**
Bagian ini menyusun urutan kolom yang diinginkan melalui variabel `order`, yaitu menggabungkan `base_cols` dengan kolom hasil preprocessing seperti original_text, clean_text, tokens, n_tokens, dan n_chars. Daftar `order` kemudian difilter agar hanya berisi kolom yang benar-benar ada di `pre_df`. Terakhir, dataframe di-reorder mengikuti urutan tersebut sehingga struktur tabel menjadi rapi dan konsisten untuk kebutuhan analisis maupun pelaporan.

In [13]:
order = base_cols + ["original_text","clean_text","tokens","n_tokens","n_chars"]
order = [c for c in order if c in pre_df.columns]
pre_df = pre_df[order]

# **Menyimpan Hasil Preprocessing**

In [14]:
OUT_CSV = "/content/traveloka_preprocessed_nltk.csv"
pre_df.to_csv(OUT_CSV, index=False)

print("Selesai")
print("Baris:", len(pre_df))
print("Kolom:", list(pre_df.columns))
pre_df.head(10)


Selesai
Baris: 56129
Kolom: ['Unnamed: 0', 'reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'original_text', 'clean_text', 'tokens', 'n_tokens', 'n_chars']


,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,original_text,clean_text,tokens,n_tokens,n_chars
0,1.0,a51b3420-eef3-4451-adf1-ba3e36acc241,Sry Haty,https://play-lh.googleusercontent.com/a/ACg8oc...,Lebih Memudahkan Untuk Memesan Tiket Saat Ingi...,5.0,0.0,3.88.0,2023-10-22 05:06:35,"Hi Kak, senang banget nih dapat rating bagus d...",2023-10-22 05:12:11,3.88.0,Lebih Memudahkan Untuk Memesan Tiket Saat Ingi...,memudahkan memesan tiket berpergian kemanapun ...,"[memudahkan, memesan, tiket, berpergian, keman...",9,81
1,2.0,fd7e4b05-854a-442c-93be-d04e79962754,Sigit Mamryanto,https://play-lh.googleusercontent.com/a/ACg8oc...,Selalu pakai Traveloka untuk pembelian tiket d...,5.0,0.0,3.88.0,2023-10-22 04:59:50,"Halo Kak, terima kasih banyak atas ratingnya, ...",2023-10-22 05:12:16,3.88.0,Selalu pakai Traveloka untuk pembelian tiket d...,pakai traveloka pembelian tiket hotel kemana,"[pakai, traveloka, pembelian, tiket, hotel, ke...",6,44
2,3.0,0dc69506-fe08-4d69-93d9-dfc3b8981393,Awawy Aljawie,https://play-lh.googleusercontent.com/a-/ALV-U...,memudahkan untuk bepergian,5.0,0.0,3.88.0,2023-10-22 04:55:44,"Hi Kak, senang banget nih dapat rating bagus d...",2023-10-22 05:12:19,3.88.0,memudahkan untuk bepergian,memudahkan bepergian,"[memudahkan, bepergian]",2,20
3,5.0,3b2b2d7a-52f0-498d-ba1e-cfe0f69eeb1c,Sumiati,https://play-lh.googleusercontent.com/a-/ALV-U...,The best app nya,5.0,0.0,3.88.0,2023-10-22 04:50:03,"Hi, thank you very much for the positive revie...",2023-10-22 05:12:25,3.88.0,The best app nya,the best app nya,"[the, best, app, nya]",4,16
4,6.0,1d5d15d5-a6d4-4e22-b429-b703ca9f4c13,Riyan Andriansyah,https://play-lh.googleusercontent.com/a/ACg8oc...,Aplikasinya bagus,5.0,0.0,NaN,2023-10-22 04:38:36,"Halo Kak, terima kasih banyak atas ratingnya, ...",2023-10-22 04:42:06,NaN,Aplikasinya bagus,aplikasinya bagus,"[aplikasinya, bagus]",2,17
5,7.0,c390c686-e774-44e1-8649-4fb2c91725d7,Ipah Rahayu,https://play-lh.googleusercontent.com/a-/ALV-U...,Untuk cari tiket akomodasi buat traveling bena...,5.0,0.0,NaN,2023-10-22 04:33:32,"Hi Kak, senang banget nih dapat rating bagus d...",2023-10-22 04:42:09,NaN,Untuk cari tiket akomodasi buat traveling bena...,cari tiket akomodasi traveling mudah pakai tra...,"[cari, tiket, akomodasi, traveling, mudah, pak...",9,65
6,8.0,1a8b72d6-d852-4a71-8c91-426b2d880e3c,Asih Karyati,https://play-lh.googleusercontent.com/a/ACg8oc...,Mempermudah kita untuk beli tiket secara onlin...,5.0,0.0,NaN,2023-10-22 04:31:44,"Hi Kak, terima kasih atas feedbacknya, semoga ...",2023-10-22 04:42:11,NaN,Mempermudah kita untuk beli tiket secara onlin...,mempermudah beli tiket online pokoknya ga ribe...,"[mempermudah, beli, tiket, online, pokoknya, g...",9,57
7,9.0,f8c90ded-8511-4c23-8143-d5c40e6202cf,Ninda Khana,https://play-lh.googleusercontent.com/a-/ALV-U...,Aplikasi andalan untuk keperluan booking tiket...,5.0,0.0,3.70.0,2023-10-22 04:31:30,"Halo Kak, terima kasih banyak atas ratingnya, ...",2023-10-22 04:42:13,3.70.0,Aplikasi andalan untuk keperluan booking tiket...,aplikasi andalan keperluan booking tiket pesaw...,"[aplikasi, andalan, keperluan, booking, tiket,...",7,54
8,10.0,52ad09c1-ba2b-4661-bb0e-6cef4f8d3cc2,William Theyer,https://play-lh.googleusercontent.com/a/ACg8oc...,Mau telepon cs aja gk bisa harus by email... P...,1.0,0.0,3.88.0,2023-10-22 03:06:29,"Hai William. Mohon maaf buat kamu kecewa, namu...",2023-10-22 03:23:33,3.88.0,Mau telepon cs aja gk bisa harus by email... P...,telepon cs gk by email refaund jadwal ganti ma...,"[telepon, cs, gk, by, email, refaund, jadwal, ...",9,52
9,11.0,5874c218-aa98-4d6e-9daf-a0347ce02362,Jonathan Cristian,https://play-lh.googleusercontent.com/a-/ALV-U...,Traveloka mantap,5.0,0.0,3.88.0,2023-10-22 02:39:46,"Halo Kak, terima kasih banyak atas ratingnya, ...",2023-10-22 02:42:08,3.88.0,Traveloka mantap,traveloka mantap,"[traveloka, mantap]",2,16


In [ ]:
from google.colab import files
files.download("/content/traveloka_preprocessed_nltk.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>